## Eigene Umgebung

Jede Umgebung ist nur eine Klasse mit Funktionen -> Eigener Prozess

In [32]:
import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO
from gymnasium.spaces import Discrete, Box
from stable_baselines3.common.env_checker import check_env
from typing import Optional

### Spaces

Container für Werte, hat aber keine konkrete Werte, sondern gibt nur an was, in dem Container drin sein könnte

Discrete: Beschreibt, das hier X verschiedene Werte enthalten sein können

Box: Gibt einen Bereich an, in welchem sich der entsprechende Wert befinden kann

In [5]:
x = Discrete(3)

In [23]:
y = Box(0, 100, shape=(3,), dtype=int)

In [24]:
y.sample()

array([ 9, 59, 26])

### Eigene Umgebung

Jetzt können wir eine eigene Umgebung definieren

Diese muss als Oberklasse die gym.Env Klasse haben und die entsprechenden Methoden implementieren (\_\_init__, step, reset, render, close)

Aufgaben:

- action_space, observation_space definieren
- State definieren
- Step definieren + Belohnung
- Reset definieren

In [67]:
class CarDriver(gym.Env):
    def __init__(self):
        self.action_space = Discrete(3)  # Die verschiedene Aktionen (Nichts, Bremsen, Beschleunigen)
        self.observation_space = Box(0, 100, dtype=np.int8)
        self.state = 40 + np.random.randint(-10, 10)  # Startgeschwindigkeit, soll zwischen 45 und 55 liegen
        self.drive_length = 60  # Zeitlimit

    def step(self, action):
        # 0: Nichts (+0km/h)
        # 1: Bremsen (-3km/h)
        # 2: Beschleunigen (+3km/h)
        if action == 1:
            self.state -= 3
        if action == 2:
            self.state += 3

        # Belohnung
        reward = 0
        if self.state >= 45 and self.state <= 55:
            reward = 1

        self.state += np.random.randint(-1, 1)  # Noise

        self.drive_length -= 1
        return np.array([self.state], dtype=np.int8), reward, self.drive_length <= 0, False, {}

    def reset(self, *, seed: Optional[int] = None, options: Optional[dict] = None):
        self.state = 40 + np.random.randint(-10, 10)  # Startgeschwindigkeit, soll zwischen 45 und 55 liegen
        self.drive_length = 60  # Zeitlimit
        return (np.array([self.state], dtype=np.int8), {})

    def render(self):
        pass

    def close(self):
        pass

In [68]:
env = CarDriver()

In [69]:
check_env(env, warn=True)

### Agent

In [71]:
durchgaenge = 10
for x in range(durchgaenge):
    state = env.reset()  # Am Anfang von jedem Durchgang die Umgebung zurücksetzen
    score = 0  # Zähler für Belohnungen
    done = False  # Zeigt ob der Agent noch im Spiel ist

    while not done:
        env.render()
        action = env.action_space.sample()  # Random Action aus dem Action Space entnehmen
        state, reward, done, term, info = env.step(action)
        # state: Der Status des Spiels (Variablen)
        # reward: Belohnung dieses steps
        # done, term: Beendet das Spiel
        # info: Extra Daten (falls vorhanden)
        score += reward

    print(f"Durchgang {x}, Score: {score}")
env.close()

Durchgang 0, Score: 3
Durchgang 1, Score: 7
Durchgang 2, Score: 18
Durchgang 3, Score: 0
Durchgang 4, Score: 3
Durchgang 5, Score: 4
Durchgang 6, Score: 0
Durchgang 7, Score: 0
Durchgang 8, Score: 0
Durchgang 9, Score: 0


### Model

In [72]:
env = CarDriver()

In [73]:
model = PPO("MlpPolicy", env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [83]:
model.learn(total_timesteps=50_000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | 39.6     |
| time/              |          |
|    fps             | 905      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | 37.7         |
| time/                   |              |
|    fps                  | 433          |
|    iterations           | 2            |
|    time_elapsed         | 9            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0031332034 |
|    clip_fraction        | 0.0897       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.851       |
|    explained_variance   | 0.243        |
|    learning_r

In [84]:
model.save("Models/CarDriver")

In [85]:
env = CarDriver()
model = PPO.load("Models/CarDriver", env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [88]:
durchgaenge = 5
for x in range(durchgaenge):
    state = env.reset()  # Am Anfang von jedem Durchgang die Umgebung zurücksetzen
    score = 0  # Zähler für Belohnungen
    done = False  # Zeigt ob der Agent noch im Spiel ist
    state = state[0]

    while not (done or term):
        # env.render()
        action, _ = model.predict(state)  # Random Action aus dem Action Space entnehmen
        if action.ndim == 1:
            action = action[0]
        
        state, reward, done, term, info = env.step(action)
        # state: Der Status des Spiels (Variablen)
        # reward: Belohnung dieses steps
        # done, term: Beendet das Spiel
        # info: Extra Daten (falls vorhanden)
        score += reward

    print(f"Durchgang {x}, Score: {score}")
env.close()

Durchgang 0, Score: 56
Durchgang 1, Score: 54
Durchgang 2, Score: 51
Durchgang 3, Score: 46
Durchgang 4, Score: 60
